In [1]:
import pandas as pd
import time
import re
import csv
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
def saveCsvHeaders(filename, data):
    with open(filename, mode='w', encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(data)
        f.close()

In [3]:
def saveCsvData(filename, data):
    with open(filename, mode='r', encoding='utf-8') as freader:
        reader = csv.reader(freader)
        next(reader, None)

        with open(filename, mode='a', encoding='utf-8') as fwriter:
            writer = csv.writer(fwriter, delimiter=';')
            writer.writerow(data)

    freader.close()
    fwriter.close()

In [4]:
URL = "https://loterias.caixa.gov.br/Paginas/Mega-Sena.aspx"
XPATH_ROWS = '//*[@id="ctl50_g_cf05b8d5_fd75_46b5_bdfa_a623e654362c"]/div/div/table/tbody'
X_PATH_THEAD = '//*[@id="ctl50_g_cf05b8d5_fd75_46b5_bdfa_a623e654362c"]/div/div/table/thead'
X_PATH_THEAD_TR = '//*[@id="ctl50_g_cf05b8d5_fd75_46b5_bdfa_a623e654362c"]/div/div/table/thead/tr'

In [5]:
option = webdriver.ChromeOptions()
option.add_argument('headless')
driver = webdriver.Chrome(options=option)
driver.get(URL)

In [6]:
driver.title

'Mega-Sena - Portal Loterias | CAIXA'

In [7]:
a = driver.find_element(By.CLASS_NAME, 'zeta')
driver.implicitly_wait(30)
driver.execute_script("arguments[0].click();", a)

In [79]:
driver.window_handles[-1]

'CDwindow-A30A80F6889DF2DCB09D18C00DB02768'

In [8]:
driver.switch_to.window(driver.window_handles[-1])

In [9]:
driver.title

'Download de Resultados - Portal Loterias | CAIXA'

In [10]:
thead = driver.find_element(By.XPATH, X_PATH_THEAD)
tr = thead.find_element(By.XPATH, X_PATH_THEAD_TR)
thead_rows = tr.find_elements(By.TAG_NAME, 'th')

In [11]:
thead_columns = []
for column in thead_rows:
    thead_columns.append(column.text)

In [106]:
saveCsvHeaders('original-data.csv', thead_columns)

In [12]:
rows = driver.find_elements(By.XPATH, XPATH_ROWS)

In [15]:
print(len(thead_columns))

21


In [236]:
# regexes = {
#     'regex':re.compile(r'(\s(\w{2}\s\-)) | (\-\s\w{2}\s\-\s\w{2}\s)'),
#     'regex1':re.compile(r'(\-{1,3})$'),
#     'regex2':re.compile(r'^(\-{1,3})'),
#     'regex3':re.compile(r'\-+'),
#     'regex4':re.compile(r'\s\-\s\w{2}'),
#     'regex5':re.compile(r'[a-z,A-Z]{2}\s\-\s[R]'),
#     'regex6':re.compile(r'\d\s\w{2}'),
#     'regex7': re.compile(r'[R]\$\d+\,\d{2}\s\w(.*)\s[R]\$\d+\,\d{2}')
# }

# def cleanTextCityOnOriginalDataset(element, regexes):
#     lines = element.text.replace('\n', ' - ')

#     stage_1 = re.search(regexes['regex'], lines)
#     if stage_1:
#         text = stage_1.group().replace(' ', '-')
#         text = re.sub(regexes['regex1'], ' ', text)
#         text = re.sub(regexes['regex2'], ' ', text)
#         text = re.sub(regexes['regex3'], '-', text)
#         lines = re.sub(regexes['regex'], text, lines)

#     stage_2 = re.search(regexes['regex4'], lines)
#     if stage_2:
#         text = stage_2.group().replace('-', '')
#         lines = re.sub(regexes['regex4'], text, lines)

#     stage_3 = re.search(regexes['regex5'], lines)
#     if stage_3:
#         text = stage_3.group().replace('-', ' ')
#         lines = re.sub(regexes['regex5'], text, lines)

#     stage_4 = re.search(regexes['regex7'], lines)
#     if stage_4:
#         text = stage_4.group()
#         print(text)

#     lines = lines.replace('  ', ' ').replace('  ', ' ')
    

#     lines_list = lines.split(' ')
#     lines_list_filtered = lines_list[:20]


#     if lines_list[21:]:
#         lines_list_filtered.append(' '.join(lines_list[21:]))
#     else:
#         lines_list_filtered.append('Sem comentários')

#     return lines_list_filtered

In [110]:
def currency_to_number(replace_dot_by_comma, remove_dot, remove_dollar, text):
    return re.sub(replace_dot_by_comma, '.', re.sub(remove_dot, '', re.sub(remove_dollar, '', text)))

In [222]:
tr = driver.find_element(By.XPATH, '//*[@id="ctl50_g_cf05b8d5_fd75_46b5_bdfa_a623e654362c"]/div/div/table/tbody[1349]/tr')

In [229]:
# terms = tr.text.replace('\n', '::',).replace(' ', ':').split('::')
# first_term = terms[0]
# last_term = terms[-1]
# middle_terms = terms[1:-1]

# city = ':'.join(middle_terms)
# new_row = []
# new_row.append(first_term.replace(':', ' '))
# new_row.append(city)
# new_row.append(last_term.replace(':', ' '))

cities = tr.find_elements(By.TAG_NAME, 'tbody')



In [256]:
lists = [None] * len(thead_rows)
remove_dot = re.compile(r'\.')
remove_dollar = re.compile(r'^[R]\$')
remove_two_words = re.compile(r'^(\w{2})$')
remove_currency = re.compile(r'^[R]\$\d+\,\d{2}$')
remove_no = re.compile(r'^(N|n)\w{2}$')
replace_dot_by_comma = re.compile('\,')


for element in rows:
    tds = element.find_elements(By.TAG_NAME, 'td')
    cities = [td.text.replace('\n', '::').replace(' ', ':') for td in tds[14].find_elements(By.TAG_NAME, 'tbody')]
    lists[0] = int(tds[0].text)
    lists[1] = datetime.strptime(tds[1].text, "%d/%m/%Y")
    lists[2] = int(tds[2].text)
    lists[3] = int(tds[3].text)
    lists[4] = int(tds[4].text)
    lists[5] = int(tds[5].text)
    lists[6] = int(tds[6].text)
    lists[7] = int(tds[7].text)
    lists[8] = tds[8].text
    lists[9] = tds[9].text
    lists[10] = tds[10].text
    lists[11] = currency_to_number(replace_dot_by_comma, remove_dot, remove_dollar, tds[11].text)
    lists[12] = currency_to_number(replace_dot_by_comma, remove_dot, remove_dollar, tds[12].text)
    lists[13] = currency_to_number(replace_dot_by_comma, remove_dot, remove_dollar, tds[13].text)
    lists[14] =  '-' if not cities[0] else cities[0].upper()
    lists[15] =  currency_to_number(replace_dot_by_comma, remove_dot, remove_dollar, 'R$0,00' if not tds[-6].text else tds[-6].text)
    lists[16] = currency_to_number(replace_dot_by_comma, remove_dot, remove_dollar, 'R$0,00' if not tds[-5].text else re.sub(remove_two_words, 'R$0,00', tds[-5].text))
    lists[17] = currency_to_number(replace_dot_by_comma, remove_dot, remove_dollar, 'R$0,00' if not tds[-4].text else tds[-4].text) 
    lists[18] = 'Sim' if not tds[-3].text else re.sub(remove_two_words, 'Não', re.sub(remove_currency, 'Não', tds[-3].text))
    lists[19] = 'Não' if not tds[-2].text else re.sub(remove_currency, 'Não', tds[-2].text)
    lists[20] = '-' if not tds[-1].text else re.sub(remove_currency, '-', re.sub(remove_no, '-', tds[-1].text))

    saveCsvData('original-data.csv', data=lists)
    
driver.quit()    

In [368]:
driver.quit()

In [277]:
base_mega_sena = pd.read_csv('original-data.csv', sep=";", encoding="utf-8")
base_mega_sena

,Concurso,Data do Sorteio,Coluna1,Coluna2,Coluna3,Coluna4,Coluna5,Coluna6,Ganhadores Faixa 1,Ganhadores Faixa 2,...,Rateio Faixa 1,Rateio Faixa2,Rateio Faixa 3,Cidade,Valor Arrecadado,Estimativa para o próximo concurso,Valor Acumulado Próximo Concurso,Acumulado,Sorteio Especial,Observação
0,1,1996-03-11 00:00:00,4,5,30,33,41,52,0,17,...,0.00,39158.92,330.21,-,0.0,0.0,1.714650e+06,Sim,Não,-
1,2,1996-03-18 00:00:00,9,37,39,41,43,49,1,65,...,2307162.23,14424.02,208.91,PR,0.0,0.0,0.000000e+00,Não,Não,-
2,3,1996-03-25 00:00:00,10,11,29,30,36,47,2,62,...,391192.51,10515.93,153.01,RN::SP,0.0,0.0,0.000000e+00,Não,Não,-
3,4,1996-04-01 00:00:00,1,5,6,27,42,59,0,39,...,0.00,15322.24,180.48,-,0.0,0.0,7.170808e+05,Sim,Não,-
4,5,1996-04-08 00:00:00,1,2,6,16,19,46,0,98,...,0.00,5318.10,96.53,-,0.0,0.0,1.342489e+06,Sim,Não,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2559,2560,2023-02-01 00:00:00,4,5,17,20,48,52,0,151,...,0.00,52127.84,837.78,-,136523016.0,135000000.0,1.168782e+08,Sim,Não,-
2560,2561,2023-02-04 00:00:00,19,22,37,44,51,56,0,393,...,0.00,23693.33,668.17,-,161502007.5,160000000.0,1.340309e+08,Sim,Não,-
2561,2562,2023-02-08 00:00:00,6,12,32,44,51,57,2,226,...,76403943.65,45102.73,992.48,PIRACICABA:SP::SAO:PAULO:SP,176795226.0,3000000.0,0.000000e+00,Não,Não,-
2562,2563,2023-02-11 00:00:00,5,9,14,30,38,50,0,49,...,0.00,54606.04,1029.19,-,46408333.5,10000000.0,4.928913e+06,Sim,Não,-


In [278]:
base_mega_sena.isnull().sum()

Concurso                              0
Data do Sorteio                       0
Coluna1                               0
Coluna2                               0
Coluna3                               0
Coluna4                               0
Coluna5                               0
Coluna6                               0
Ganhadores Faixa 1                    0
Ganhadores Faixa 2                    0
Ganhadores Faixa 3                    0
Rateio Faixa 1                        0
Rateio Faixa2                         0
Rateio Faixa 3                        0
Cidade                                0
Valor Arrecadado                      0
Estimativa para o próximo concurso    0
Valor Acumulado Próximo Concurso      0
Acumulado                             0
Sorteio Especial                      0
Observação                            0
dtype: int64

In [279]:
base_mega_sena.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564 entries, 0 to 2563
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Concurso                            2564 non-null   int64  
 1   Data do Sorteio                     2564 non-null   object 
 2   Coluna1                             2564 non-null   int64  
 3   Coluna2                             2564 non-null   int64  
 4   Coluna3                             2564 non-null   int64  
 5   Coluna4                             2564 non-null   int64  
 6   Coluna5                             2564 non-null   int64  
 7   Coluna6                             2564 non-null   int64  
 8   Ganhadores Faixa 1                  2564 non-null   int64  
 9   Ganhadores Faixa 2                  2564 non-null   int64  
 10  Ganhadores Faixa 3                  2564 non-null   int64  
 11  Rateio Faixa 1                      2564 no

In [281]:
base_mega_filtered = base_mega_sena.loc[:, base_mega_sena.columns.intersection(['Data do Sorteio', 'Coluna1', 'Coluna2','Coluna3','Coluna4','Coluna5','Coluna6','Ganhadores Faixa 1'])]
base_mega_filtered['Class'] = base_mega_sena['Ganhadores Faixa 1'] > 0
base_mega_filtered['Class'] = base_mega_filtered['Class'].astype(int)
base_mega_filtered

,Data do Sorteio,Coluna1,Coluna2,Coluna3,Coluna4,Coluna5,Coluna6,Ganhadores Faixa 1,Class
0,1996-03-11 00:00:00,4,5,30,33,41,52,0,0
1,1996-03-18 00:00:00,9,37,39,41,43,49,1,1
2,1996-03-25 00:00:00,10,11,29,30,36,47,2,1
3,1996-04-01 00:00:00,1,5,6,27,42,59,0,0
4,1996-04-08 00:00:00,1,2,6,16,19,46,0,0
...,...,...,...,...,...,...,...,...,...
2559,2023-02-01 00:00:00,4,5,17,20,48,52,0,0
2560,2023-02-04 00:00:00,19,22,37,44,51,56,0,0
2561,2023-02-08 00:00:00,6,12,32,44,51,57,2,1
2562,2023-02-11 00:00:00,5,9,14,30,38,50,0,0


In [283]:
base_mega_filtered.query("Class == 1")

,Data do Sorteio,Coluna1,Coluna2,Coluna3,Coluna4,Coluna5,Coluna6,Ganhadores Faixa 1,Class
1,1996-03-18 00:00:00,9,37,39,41,43,49,1,1
2,1996-03-25 00:00:00,10,11,29,30,36,47,2,1
10,1996-05-20 00:00:00,15,25,37,38,58,59,1,1
16,1996-07-01 00:00:00,6,10,13,19,20,51,1,1
23,1996-08-19 00:00:00,1,8,14,28,33,43,1,1
...,...,...,...,...,...,...,...,...,...
2533,2022-10-29 00:00:00,28,36,39,44,56,60,1,1
2536,2022-11-09 00:00:00,12,24,26,31,37,48,1,1
2547,2022-12-14 00:00:00,9,15,23,25,29,30,1,1
2549,2022-12-31 00:00:00,4,5,10,34,58,59,5,1
